### Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
import matplotlib.colors as mcolors
import subjects

### Firing rate raster (5 minute windows) PN, IN and SWA

In [ ]:
fig = plotting.Fig(grid=(8, 9),constrained_layout=False)
subfig = fig.add_subfigure(fig.gs[:2, 1:3])
axs = subfig.subplots(6, 1,sharex=True, gridspec_kw=dict(height_ratios=[1,2,5, 1,2,5]))

swa = subjects.GroupData().swa_examples

sessions = subjects.nsd.ratUday2 + subjects.sd.ratUday4
# sessions = subjects.nsd.ratSday2 + subjects.sd.ratSday3

for s, sess in enumerate(sessions):
    neurons = sess.neurons_stable.get_neuron_type(["pyr", "inter"])
    mean_frate = neurons.firing_rate
    neurons = neurons[np.argsort(mean_frate)]
    n_neurons = neurons.n_neurons

    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    pre = [np.max([pre[1] - 3600, pre[0]]), pre[1]]
    post1 = [post[0], post[0] + 8 * 3600]
    # post2 = [post[0] + 2.5 * 3600, post[0] + 5 * 3600]
    # post3 = [post[0] + 5 * 3600, post[0] + 7.5 * 3600]

    # periods = [pre, maze, post1, post2, post3]
    periods = [post1]

    swa_amp = swa['swa'][s]
    swa_time = swa['swa_time'][s]

    for i, p in enumerate(periods):
        binspk = neurons.time_slice(*p).get_binned_spiketrains(bin_size=5*60)

        ax = axs[3 * s]
        plotting.plot_epochs(
            sess.brainstates.time_slice(*p),
            ax=ax,
            labels_order=["NREM", "REM", "QW", "AW"],
            colors=subjects.colors_sleep,
        )
        ax.set_axis_off()

        ax = axs[3*s+1]
        ax.fill_between(swa_time,0,swa_amp*100,color='#0773ed',lw=0.5,ec=None)
        ax.set_ylim(20,450)
        ax.set_yticks([100,250,400])

        ax = axs[3 * s + 2]
        im=ax.pcolormesh(
            binspk.time+2.5*60,
            np.arange(neurons.n_neurons) + 1,
            binspk.firing_rate,
            cmap="plasma",
            norm=mcolors.LogNorm(0.02, 15, clip=True),
            rasterized=True,
        )
        ax.set_yticks([1,n_neurons//2,n_neurons])
        ax.set_ylabel('Neurons')
        xticks = np.arange(p[0],p[1],2.5*3600)
        ax.set_xticks(xticks,(xticks-p[0])/3600)

        cax = fig.subplot(fig.gs[1,4])
        plt.colorbar(im,ax=cax,shrink=0.5)

        ax = ax.twinx()
        ax.plot(binspk.time,np.mean(binspk.firing_rate,axis=0),'w',lw=0.8)
        ax.set_ylim(1,6)
        ax.set_ylabel('Mean firing rate (Hz)')
        fig.toggle_spines(ax,sides=['right'],keep=True)

        
        # ax.set_facecolor("black")

fig.savefig(subjects.figpath_sd/'frate_example')

### Firing rate raster (5 minute windows) with session frate and mean frate across all

In [ ]:

slp_states = sess.brainstates.time_slice(*p).to_dataframe()
slp_states.loc[slp_states.label.isin(['AW','QW']),'label'] = 'W'



In [ ]:
slp_states

In [ ]:
from neuropy.core import Epoch

fig = plotting.Fig(grid=(8, 9),constrained_layout=False)
subfig = fig.add_subfigure(fig.gs[:2, 1:3])
axs = subfig.subplots(4, 1,sharex=True, gridspec_kw=dict(height_ratios=[1,3, 1,3]))


sessions = subjects.nsd.ratUday2 + subjects.sd.ratUday4

frate = subjects.GroupData().frate_ratio_nsd_vs_sd
mean_frate = frate[frate.neuron_type=='pyr'].groupby(['grp','zt']).mean().reset_index()
mean_frate = mean_frate[(mean_frate.zt>=0)&(mean_frate.zt<=8)]
nsd_mean_frate = mean_frate[mean_frate.grp=='NSD']['frate']
sd_mean_frate = mean_frate[mean_frate.grp=='SD']['frate']

grp_mean_frate = [nsd_mean_frate,sd_mean_frate]
# sessions = subjects.nsd.ratSday2 + subjects.sd.ratSday3

for s, sess in enumerate(sessions):
    neurons = sess.neurons_stable.get_neuron_type(["pyr"])
    mean_frate = neurons.firing_rate
    neurons = neurons[np.argsort(mean_frate)]
    n_neurons = neurons.n_neurons

    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    pre = [np.max([pre[1] - 3600, pre[0]]), pre[1]]
    post1 = [post[0], post[0] + 8 * 3600]
    # post2 = [post[0] + 2.5 * 3600, post[0] + 5 * 3600]
    # post3 = [post[0] + 5 * 3600, post[0] + 7.5 * 3600]

    # periods = [pre, maze, post1, post2, post3]
    periods = [post1]

    # swa_amp = swa['swa'][s]
    # swa_time = swa['swa_time'][s]

    for i, p in enumerate(periods):
        binspk = neurons.time_slice(*p).get_binned_spiketrains(bin_size=5*60)

        ax = axs[2 * s]
        slp_states = sess.brainstates.time_slice(*p).to_dataframe()
        slp_states.loc[slp_states.label.isin(['AW','QW']),'label'] = 'W'
        slp_states = Epoch(slp_states).merge_neighbors()

        plotting.plot_epochs(
            slp_states,
            ax=ax,
            labels_order=["NREM", "REM", "W"],
            colors='k',
        )
        ax.set_axis_off()

        # ax = axs[3*s+1]
        # ax.fill_between(swa_time,0,swa_amp*100,color='#0773ed',lw=0.5,ec=None)
        # ax.set_ylim(20,450)
        # ax.set_yticks([100,250,400])

        ax = axs[2 * s + 1]
        im=ax.pcolormesh(
            binspk.time+2.5*60,
            np.arange(neurons.n_neurons) + 1,
            binspk.firing_rate,
            cmap="plasma",
            norm=mcolors.LogNorm(0.02, 15, clip=True),
            rasterized=True,
        )
        ax.set_yticks([1,n_neurons//2,n_neurons])
        ax.set_ylabel('Neurons')
        xticks = np.arange(p[0],p[1],2.5*3600)
        ax.set_xticks(xticks,(xticks-p[0])/3600)

        cax = fig.subplot(fig.gs[1,4])
        plt.colorbar(im,ax=cax,shrink=0.5)

        ax = ax.twinx()
        ax.plot(binspk.time,np.mean(binspk.firing_rate,axis=0),'w',lw=0.8)
        ax.plot(binspk.time,grp_mean_frate[s],'w',lw=0.8)
        ax.set_ylim(0.2, 1.5)
        ax.set_ylabel('Mean firing rate (Hz)')
        fig.toggle_spines(ax,sides=['right'],keep=True)

        
        # ax.set_facecolor("black")

fig.savefig(subjects.figpath_sd/'frate_example')

### Firing rate raster (5 minute windows) with PN and IN in separate panels

In [ ]:
fig = plotting.Fig(grid=(8, 9), constrained_layout=False)
subfig = fig.add_subfigure(fig.gs[:2, 1:3])
axs = subfig.subplots(
    4, 1, sharex=True, gridspec_kw=dict(height_ratios=[1, 3, 1, 3])
)


sessions = subjects.nsd.ratUday2 + subjects.sd.ratUday4
# sessions = subjects.nsd.ratSday2 + subjects.sd.ratSday3

for s, sess in enumerate(sessions):
    neurons = sess.neurons_stable.get_neuron_type(["pyr", "inter"])
    mean_frate = neurons.firing_rate
    neurons = neurons[np.argsort(mean_frate)]

    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()

    pre = [np.max([pre[1] - 3600, pre[0]]), pre[1]]
    post1 = [post[0], post[0] + 8 * 3600]

    ax = axs[2 * s]
    plotting.plot_epochs(
        sess.brainstates.time_slice(*post1),
        ax=ax,
        labels_order=["NREM", "REM", "QW", "AW"],
        colors=subjects.colors_sleep,
    )
    ax.set_axis_off()

    for i, ntype in enumerate(["pyr"]):
        binspk = (
            neurons.get_neuron_type(ntype)
            .time_slice(*post1)
            .get_binned_spiketrains(bin_size=5 * 60)
        )
        n_neurons = binspk.n_neurons

        if ntype=='pyr':
            norm=mcolors.LogNorm(0.02, 15, clip=True)
        if ntype == 'inter':
            norm=mcolors.LogNorm(10, 40, clip=True)


        ax = axs[2 * s + i + 1]
        im = ax.pcolormesh(
            binspk.time + 2.5 * 60,
            np.arange(binspk.n_neurons) + 1,
            binspk.firing_rate,
            cmap="plasma",
            norm=norm,
            rasterized=True,
        )
        ax.set_yticks([1, n_neurons // 2, n_neurons])
        ax.set_ylabel("Neurons")
        xticks = np.arange(post1[0], post1[1], 2.5 * 3600)
        ax.set_xticks(xticks, (xticks - post1[0]) / 3600)

        cax = fig.subplot(fig.gs[1, 4])
        plt.colorbar(im, ax=cax, shrink=0.5)

        ax = ax.twinx()
        ax.plot(binspk.time, np.mean(binspk.firing_rate, axis=0), "w", lw=0.8)
        ax.set_ylim(0.2, 1.5)
        ax.set_ylabel("Mean firing rate (Hz)")
        fig.toggle_spines(ax, sides=["right"], keep=True)

        # ax.set_facecolor("black")

fig.savefig(subjects.figpath_sd/'frate_pyr_example')
